# Document Ingestion with Unstructured

This notebook explores document ingestion using the Unstructured library.

In [ ]:
# Install dependencies if needed
# !pip install unstructured[pdf]

In [1]:
from unstructured.partition.pdf import partition_pdf
import os

2026-02-07 21:33:06.521638640 [W:onnxruntime:Default, device_discovery.cc:164 DiscoverDevicesForPlatform] GPU device discovery failed: device_discovery.cc:89 ReadFileContents Failed to open file: "/sys/class/drm/card0/device/vendor"
/home/zord/learn-rag/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set the PDF path
pdf_path = "../docs/DocLayNet.pdf"
print(f"Processing: {pdf_path}")
print(f"File exists: {os.path.exists(pdf_path)}")

Processing: ../docs/DocLayNet.pdf
File exists: True


In [3]:
# Extract elements
elements = partition_pdf(
    filename=pdf_path,
    strategy="hi_res",
    infer_table_structure=True, # infer table structure
    extract_image_block_types=["Image","Table"], # specify block types to extract
    extract_image_block_to_payload=True, # include image data in element payload metadata
    # chunking_strategy="by_title",
    # extract_images_in_pdf=True, # extract images
    # extract_image_block_output_dir="figures_test", # specify output dir for extracted images
)

print(f"Extracted {len(elements)} elements")

The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


Extracted 198 elements


In [4]:
import json

print(json.dumps([el.to_dict() for el in elements], indent=2))

[
  {
    "type": "UncategorizedText",
    "element_id": "e62ff842ae310c5e11c68e1b15e08c25",
    "text": "2",
    "metadata": {
      "coordinates": {
        "points": [
          [
            45.388888888888886,
            606.5555555555555
          ],
          [
            45.388888888888886,
            634.3333333333334
          ],
          [
            100.94444444444446,
            634.3333333333334
          ],
          [
            100.94444444444446,
            606.5555555555555
          ]
        ],
        "system": "PixelSpace",
        "layout_width": 1700,
        "layout_height": 2200
      },
      "last_modified": "2026-02-07T17:51:40",
      "filetype": "application/pdf",
      "languages": [
        "eng"
      ],
      "page_number": 1,
      "file_directory": "../docs",
      "filename": "DocLayNet.pdf"
    }
  },
  {
    "type": "UncategorizedText",
    "element_id": "9a611a955ba63f5bba09222895c5b73a",
    "text": "2022",
    "metadata": {
      "coo

##### Summary: partition_pdf() Parameters

| Parameter                        | Type      | What It Does                                                                                                                                                      |
| -------------------------------- | --------- | ----------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| `filename`                       | str       | **(Required)** Path to the PDF file to eat.                                                                                                                       |
| `strategy`                       | str       | Which parsing strategy to use: `"auto"`, `"fast"`, `"hi_res"`, `"ocr_only"`. Controls speed, accuracy, and table/image support.([unstructured.readthedocs.io][1]) |
| `infer_table_structure`          | bool      | If **True**, enables structured table extraction, including additional metadata like HTML output.([unstructured.readthedocs.io][1])                               |
| `extract_image_block_types`      | list[str] | Tell the parser which element types to extract as **image blocks**, such as `["Image", "Table"]`.([unstructured.readthedocs.io][1])                               |
| `extract_image_block_to_payload` | bool      | If **True**, includes image data (Base64) in each element’s metadata instead of just saving external files.([unstructured.readthedocs.io][1])                     |
| `extract_image_block_output_dir` | str       | Directory to save extracted images if not embedding them.([unstructured.readthedocs.io][1])                                                                       |
| `languages`                      | list[str] | Languages for OCR (Tesseract). Required if doing OCR on scanned PDFs.([unstructured.readthedocs.io][1])                                                           |
| `encoding`                       | str       | Text encoding to use when reading text (defaults to utf-8).([unstructured.readthedocs.io][1])                                                                     |
| `include_page_breaks`            | bool      | If **True**, inserts `PageBreak` elements in output where pages break.([unstructured.readthedocs.io][1])                                                          |
| `start_page_number`              | int       | Set the **first page number** (useful for multi-document processing).([unstructured.readthedocs.io][1])                                                           |
| `skip_infer_table_types`         | list[str] | List of doc types to **skip table extraction** for.([docs.unstructured.io][2])                                                                                    |
| `hi_res_model_name`              | str       | Model name used by `hi_res` strategy (layout detection model).([docs.unstructured.io][2])                                                                         |

[1]: https://unstructured.readthedocs.io/en/latest/core/partition.html?utm_source=chatgpt.com "Partitioning - Unstructured 0.12.6 documentation"
[2]: https://docs.unstructured.io/api-reference/api-services/api-parameters?utm_source=chatgpt.com "Partition Endpoint parameters - Unstructured"


##### When to Use Each Option

| Goal                            | Settings                                           |
| ------------------------------- | -------------------------------------------------- |
| Extract tables as structured    | `strategy="hi_res"`, `infer_table_structure=True`  |
| Extract embedded PDF images     | `extract_image_block_types=["Image"]`              |
| Embed images directly in return | `extract_image_block_to_payload=True`              |
| Save images to folder           | `extract_image_block_output_dir="path"`            |
| OCR non-text PDFs               | `strategy="ocr_only"` or `"hi_res"` with languages |


How Images & Tables Show Up

When you tell partition_pdf() to extract image blocks:

elements = partition_pdf(
    filename="doc.pdf",
    strategy="hi_res",
    extract_image_block_types=["Image","Table"]
)


Then elements of type "Image" or "Table" include extra metadata:

📌 For Images

el.type == "Image"

You’ll find Base64 of the image in:

el.metadata["image_base64"]


The MIME type may also be in:

el.metadata["image_mime_type"]


You can decode that base64 to get the actual image bytes.

In [ ]:
# Display elements
for i, el in enumerate(elements[:10]):  # Show first 10
    print(f"{i+1}. Category: {el.category}")
    print(f"   Text: {el.text[:200]}...")
    # print(el)
    print("===============================================")
    print()

In [ ]:
# Separate text and tables
text_blocks = [el for el in elements if el.category != "Table"]
tables = [el for el in elements if el.category == "Table"]

print(f"Text blocks: {len(text_blocks)}")
print(f"Tables: {len(tables)}")

In [34]:
# Filtering Elements

images = [el for el in elements if el.category == "Image"]
tables = [el for el in elements if el.category == "Table"]

print("================= Images ================")
for img in images:
    img_dict = img.to_dict()
    print(img_dict['metadata']['image_base64'])
    # print("Image element:", img.element_id)
    # print("Metadata:", img.metadata)

print("================= Tables ================")
for table in tables:
    table_dict = table.to_dict()
    print(table_dict['metadata']['text_as_html'])
    # print(table['metadata']['text_as_html'])
    # print("Table text:", table.text)
    # print("HTML:", table.metadata.get("text_as_html"))


================= Images ================
...
================= Tables ================
<table><thead><tr><th rowspan="2">class label</th><th colspan="4">% of Total</th><th colspan="7">triple inter-annotator mAP @ 0.5-0.95 (%)</th></tr><tr><th>Count |</th><th>Train</th><th>Test</th><th>Val</th><th>All</th><th>Fin</th><th>Man</th><th>Sci</th><th>Law</th><th>Pat</th><th>Ten</th></tr></thead><tbody><tr><td>Caption</td><td>22524</td><td>2.04</td><td>1.77</td><td>2.32</td><td>| 84-89</td><td>40-61</td><td>86-92</td><td>94-99</td><td>95-99</td><td>69-78</td><td>n/a</td></tr><tr><td>Footnote</td><td>6318</td><td>0.60</td><td>0.31</td><td>0.58</td><td>| 83-91</td><td>n/a</td><td>100</td><td>62-88</td><td>85-94</td><td>n/a</td><td>82-97</td></tr><tr><td>Formula</td><td>25027</td><td>2.25</td><td>1.90</td><td>2.96</td><td>| 83-85</td><td>n/a</td><td>n/a</td><td>84-87</td><td>86-96</td><td>n/a</td><td>n/a</td></tr><tr><td>List-item</td><td>185660</td><td>17.19</td><td>13.34</td><td>15.82 |</td><t